In [1]:
!git clone https://github.com/EmanueleCeglia/PDF-Metadata-Extractor.git

Cloning into 'PDF-Metadata-Extractor'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 31 (delta 7), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (31/31), 461.01 KiB | 1.96 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
%cd PDF-Metadata-Extractor

/content/PDF-Metadata-Extractor


In [ ]:
!pip install -r requirements.txt

In [ ]:
!apt-get install ghostscript -y

## Extract Dates

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from pdf_dates_finder import PDFDatesFinder
import os
import re
from tqdm import tqdm

Download model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B")
model = AutoModelForCausalLM.from_pretrained("Open-Orca/OpenOrca-Platypus2-13B", device_map="auto", load_in_8bit=True)

Iterate on each insurance policy and apply prompt request

In [10]:
def post_process_response_dates(text):
    start_date_pattern = r"Start date:\s*([0-9]+[a-z]*\s+[A-Z][a-z]+\s+[0-9]{4})"
    end_date_pattern = r"End date:\s*([0-9]+[a-z]*\s+[A-Z][a-z]+\s+[0-9]{4})"

    start_date_match = re.search(start_date_pattern, text)
    end_date_match = re.search(end_date_pattern, text)

    result = {}
    if start_date_match and end_date_match:
        result['Start date'] = start_date_match.group(1)
        result['End date'] = end_date_match.group(1)
    else:
        return None

    return result

Dates extraction

In [11]:
insurances_folder_path = "/content/PDF-Metadata-Extractor/Insurances"

extracted_dates = {}

for root, dirs, files in os.walk(insurances_folder_path):
    for file_name in tqdm(files):
        full_file_path = os.path.join(root, file_name)

        extraction = PDFDatesFinder(full_file_path)
        extraction.load_pdf()
        extraction.process_text()
        dates = extraction.find_dates()

        for value in dates.values():
          for phrase in value.values():
            if len(phrase):
              sentence = phrase[0]

              prompt = "Find start date and end date from the following sentence: " + sentence
              inputs = tokenizer(prompt, return_tensors="pt")
              generate_ids = model.generate(inputs.input_ids, max_length=1000)
              response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
              response = post_process_response_dates(response)

              if response:
                extracted_dates[str(file_name)] = response
                print(str(file_name) + ' EXTRACTED!')
              else:
                print(str(file_name) + ' NOT FOUND!')

  0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


policy_8.pdf NOT FOUND!
policy_8.pdf NOT FOUND!
policy_8.pdf NOT FOUND!


 14%|█▍        | 1/7 [21:14<2:07:24, 1274.15s/it]

policy_8.pdf NOT FOUND!


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


...

policy_9.pdf NOT FOUND!


 86%|████████▌ | 6/7 [43:35<05:34, 334.49s/it]

policy_9.pdf EXTRACTED!


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
100%|██████████| 7/7 [46:17<00:00, 396.84s/it]

policy_5.pdf EXTRACTED!


In [14]:
print(extracted_dates)

{'policy_2.pdf': {'Start date': '23 November 2020', 'End date': '15 December 2020'}, 'policy_4.pdf': {'Start date': '1st May 2022', 'End date': '30th April 2023'}, 'policy_1.pdf': {'Start date': '31st March 2021', 'End date': '30th March 2022'}, 'policy_10.pdf': {'Start date': '06th December 2021', 'End date': '10th December 2021'}, 'policy_9.pdf': {'Start date': '1 June 2022', 'End date': '1 December 2023'}, 'policy_5.pdf': {'Start date': '06 December 2021', 'End date': '06 December 2021'}}
